In [1]:
from kaggle_environments import make
import matplotlib.colors as colors
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path
import sys
import torch
import tqdm
from typing import *

#from lux_ai.lux.game import Game
from lux_ai.lux_gym.lux_env import LuxEnv
from lux_ai.rl_agent.rl_agent import RLAgent

#env = make("lux_ai_2021", configuration={"seed": 56221, "loglevel": 2}, debug=True)
env = make("lux_ai_2021", configuration={"loglevel": 2}, debug=True)

rl_agent = RLAgent(env.steps[0][0]["observation"], env.configuration)

Loading environment football failed: No module named 'gfootball'


In [3]:
mu_1 = 35.838
mu_2 = 29.581
BETA = 4. + 1. / 6.

def win_probability(mu_1, mu_2):
    exp = (mu_1 - mu_2) / BETA
    n = 4. ** exp
    return n / (n + 1)

win_probability(mu_1, mu_2)

0.889118702815182

In [2]:
env.reset()
env.run(["/home/isaiah/GitHub/Kaggle/Lux_AI_2021/main.py", "/home/isaiah/GitHub/Kaggle/luxai2021_handcrafted_agents/agents/main_coral.py"])

Turn: 1 - Predicted value: -0.03 - Timing info: {Observation processing: 0.02 - Model inference: 0.95 - Collision detection: 0.01} - Remaining overage time: 60.00
Turn: 2 - Predicted value: 0.15 - Timing info: {Observation processing: 0.07 - Model inference: 0.10 - Collision detection: 0.01} - Remaining overage time: 60.00
Turn: 3 - Predicted value: 0.15 - Timing info: {Observation processing: 0.08 - Model inference: 0.15 - Collision detection: 0.01} - Remaining overage time: 60.00
Turn: 4 - Predicted value: 0.16 - Timing info: {Observation processing: 0.07 - Model inference: 0.14 - Collision detection: 0.01} - Remaining overage time: 60.00
Turn: 5 - Predicted value: 0.12 - Timing info: {Observation processing: 0.02 - Model inference: 0.14 - Collision detection: 0.01} - Remaining overage time: 60.00
Turn: 6 - Predicted value: 0.16 - Timing info: {Observation processing: 0.02 - Model inference: 0.14 - Collision detection: 0.01} - Remaining overage time: 60.00
Turn: 7 - Predicted value: 

[[{'action': [],
   'reward': 0,
   'info': {},
   'observation': {'remainingOverageTime': 60,
    'step': 0,
    'width': 16,
    'height': 16,
    'reward': 0,
    'globalUnitIDCount': 2,
    'globalCityIDCount': 2,
    'player': 0,
    'updates': ['0',
     '16 16',
     'rp 0 0',
     'rp 1 0',
     'r coal 0 0 406',
     'r coal 0 15 406',
     'r wood 1 7 351',
     'r wood 1 8 351',
     'r wood 2 5 347',
     'r wood 2 6 346',
     'r wood 2 7 389',
     'r wood 2 8 389',
     'r wood 2 9 346',
     'r wood 2 10 347',
     'r wood 3 5 399',
     'r wood 3 6 302',
     'r wood 3 9 302',
     'r wood 3 10 399',
     'r coal 6 0 414',
     'r coal 6 15 414',
     'r wood 9 7 395',
     'r wood 9 8 395',
     'r wood 10 6 800',
     'r wood 10 7 389',
     'r wood 10 8 389',
     'r wood 10 9 800',
     'r wood 11 0 377',
     'r wood 11 6 800',
     'r wood 11 9 800',
     'r wood 11 15 377',
     'r wood 12 0 354',
     'r wood 12 6 800',
     'r wood 12 9 800',
     'r wood 12 1

In [3]:
env.render(mode="ipython", height=1000, width=1200)

In [1]:
import json
import numpy as np
import pandas as pd
from pathlib import Path

pd.set_option("display.precision", 2)

In [2]:
def extract_results_to_dict(
    replay_file: Path,
    points_for_win: float = 1.,
    points_for_loss: float = 0.,
    points_for_tie: float = 0.5,
    verbose: bool = True,
) -> dict:
    try:
        with open(replay_file, "r") as f:
            replay = json.load(f)
    except json.JSONDecodeError as e:
        if verbose:
            print(f"JSONDecodeError for file {replay_file}: {e}")
        return {}
    
    agent_names = [a["name"] for a in replay["teamDetails"]]
    # Check if the match was a draw
    if replay["results"]["ranks"][0]["rank"] == replay["results"]["ranks"][1]["rank"]:
        agent_points = [points_for_tie, points_for_tie]
    else:
        agent_points = {
            result["agentID"]: points_for_win if result["rank"] == 1 else points_for_loss
            for result in replay["results"]["ranks"]
        }
    
    assert replay["width"] == replay["height"]
    map_size = replay["width"]
    return {
        "map_size": map_size,
        "p1": agent_names[0],
        "p2": agent_names[1],
        "p1_points": agent_points[0],
        "p2_points": agent_points[1],
    }

In [31]:
replay_dirs = [
    "/home/isaiah/lux_ai/RL_main__vs__09-17_22-05-30_20000128_main/",
    "/home/isaiah/lux_ai/RL_main__vs__10-02_11-29-02_20000192_main",
    "/home/isaiah/lux_ai/RL_main__vs__RL_main_2/"
]


all_result_dfs = {}
for replays_dir in replay_dirs:
    all_results = pd.DataFrame([extract_results_to_dict(file_name, verbose=False) for file_name in sorted(Path(replays_dir).glob("*.json"))]).dropna()
    if all_results.shape[0] == 0 or all_results.shape[1] == 0:
        print(f"Skipping {replays_dir}")
        continue
    agent_point_cols = []
    for agent_name in pd.unique(all_results[["p1", "p2"]].values.ravel()):
        apc = f"{agent_name}_points"
        agent_point_cols.append(apc)
        all_results[apc] = all_results["p1_points"].where(all_results["p1"] == agent_name, all_results["p2_points"])

    df_subset = all_results[["map_size", *agent_point_cols]]
    result = df_subset.groupby("map_size", as_index=False).sum()
    aggregate_result = df_subset.drop(["map_size"], axis=1).sum()
    aggregate_result["map_size"] = "ALL"

    result = result.append(aggregate_result, ignore_index=True).set_index("map_size")
    all_result_dfs[replays_dir] = result

for replay_dir, df in all_result_dfs.items():
    display(df.style.set_caption(Path(replay_dir).stem))
    #display(df)

Skipping /home/isaiah/lux_ai/RL_main__vs__RL_main_2/


,RL/main.py_points,HOF/09-17_22-05-30_20000128/main.py_points
map_size,,
12.0,9.00,0.00
16.0,7.00,2.00
24.0,8.00,1.00
32.0,8.00,1.00
ALL,32.00,4.00


,RL/main.py_points,HOF/10-02_11-29-02_20000192/main.py_points
map_size,,
12.0,1.00,0.00
ALL,1.00,0.00
